In [1]:
setwd("/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM")

source("code/enrichment_fxns.R")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last




Here I perform enrichment analysis to find modules enriched for cell type markers. 

These modules will later be used to correlate with exon PSI to define cell type-specific exons.

In [2]:
network_dir <- "tasic_2018_ALM_STAR_20pcntCells_30pcntVar_200samples_log2_pseudobulk_Modules"
network_dataset <- "20pcntCells_30pcntVar_200samples_log2_pseudobulk"

In [3]:
mod_def <- "PosFDR"
unique <- FALSE
lfc_threshold <- 5

### Prep DE genes

#### 1 vs. pooled tests

In [4]:
pooled_res_list <- readRDS("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_1_vs_pooled_DE_genes_dream.RDS")
pooled_ctype_genes <- prep_DE_genes(pooled_res_list, lfc_threshold, pairwise=FALSE, unique=unique)

#### Pairwise tests

In [5]:
pairwise_res_list <- readRDS("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_pairwise_DE_genes_dream.RDS")
pairwise_ctype_genes <- prep_DE_genes(pairwise_res_list, lfc_threshold, pairwise=TRUE, unique=unique)

In [6]:
pairwise_ctype_genes

$Astro
  [1] "Ttyh1"         "Kcnk1"         "Dtna"          "Gabrb1"       
  [5] "Ntsr2"         "Cadm1"         "Gpr37l1"       "Cspg5"        
  [9] "Cldn10"        "Bcan"          "Slc1a3"        "Gja1"         
 [13] "Plpp3"         "Mt1"           "Scg3"          "Mt2"          
 [17] "Ugp2"          "F3"            "Phkg1"         "S1pr1"        
 [21] "Gjb6"          "Fgfr3"         "Nkain4"        "Acsl3"        
 [25] "Slc1a2"        "Ptprz1"        "Gm6145"        "Gm45774"      
 [29] "Arhgap5"       "Smpdl3a"       "Kcnn2"         "Aqp4"         
 [33] "Slc39a12"      "Lcat"          "Sox2"          "Acsbg1"       
 [37] "Hacd2"         "Cyp4f15"       "Idi1"          "Nwd1"         
 [41] "Prodh"         "Mmd2"          "Glul"          "Ttyh3"        
 [45] "Clu"           "Slc6a11"       "Mlc1"          "Abhd3"        
 [49] "Dio2"          "Grina"         "Nat8f1"        "Msmo1"        
 [53] "Dbx2"          "Pla2g7"        "Cbs"           "Slc25a18"     
 [57] "Hepacam"       "Acsl6"         "Paqr7"         "Fam107a"      
 [61] "Oaf"           "Tril"          "Atp13a4"       "Hes5"         
 [65] "Rmst"          "Grm3"          "Daam2"         "Itih3"        
 [69] "Btbd17"        "Adgrg1"        "Dhcr7"         "Aldoc"        
 [73] "Frmd4a"        "Agpat5"        "Grin2c"        "Fads2"        
 [77] "Mgll"          "Etnppl"        "Sfxn5"         "Gldc"         
 [81] "Gpam"          "Ppp1r3g"       "AW047730"      "Paqr8"        
 [85] "Slitrk2"       "4933431E20Rik" "Gm45875"       "Eva1a"        
 [89] "Myorg"         "Chrdl1"        "Prex2"         "Lix1"         
 [93] "Gm7908"        "Plcd4"         "Sox9"          "Lfng"         
 [97] "Pou3f3"        "Rfx4"          "Slc4a4"        "Sdc4"         
[101] "Gstm5"         "Adhfe1"        "Prdx6"         "Slc7a10"      
[105] "Timp4"         "Gnao1"         "Il18"          "Aldh1l1"      
[109] "Ednrb"         "Tlcd1"         "Pou3f2"        "9430041J12Rik"
[113] "Cdh19"         "Fam181b"       "Bmpr1b"        "Gpc5"         
[117] "Pbxip1"        "Tst"           "Gpt"           "Gm15601"      
[121] "St3gal4"       "Ckb"           "Fam228b"       "Mvd"          
[125] "Ephx2"         "Fam20a"        "Tlr3"          "Mcur1"        
[129] "Gli3"          "Gm12892"       "Nrarp"         "AI849053"     
[133] "Mt3"           "2900052N01Rik" "Clmn"          "Soat1"        
[137] "Ppil6"         "Crot"          "Pax6"          "Ppp1r3c"      
[141] "Scara3"        "Acss1"         "Gm38287"       "Gale"         
[145] "Gm9934"        "Kifc3"         "Gm14493"       "Pdlim4"       
[149] "Apln"          "Rapgef3"       "Fbrsl1"        "Ndrg2"        
[153] "Sema4b"        "Mfap3l"        "Rftn2"         "Igdcc4"       
[157] "Mid1ip1"       "Gm57100"       "Lgi4"          "Irak2"        
[161] "Sumf2"         "Slc15a2"       "Gm20421"       "Mertk"        
[165] "Etv4"          "Abcb9"         "Ndp"           "Gm15417"      
[169] "Naaa"          "Rsrp1"         "Cntfr"         "Gm44257"      
[173] "Tom1l1"        "Ift27"         "Itgav"         "Pdzph1"       
[177] "Slc27a1"       "Ghr"           "Cth"           "Serhl"        
[181] "Gabrg1"        "Gm43322"       "Gm3764"        "Gm57288"      
[185] "Fgfrl1"        "Gm13872"       "Chi3l1"        "Mpv17l2"      
[189] "Utp14b"        "Hopxos"        "Kirrel2"       "Phka1"        
[193] "Celrr"         "Cyp4f14"       "Pantr2"        "Nadk2"        
[197] "Washc2"        "Prr5"          "Mro"           "C030018K13Rik"
[201] "Pld2"          "Htra1"         "Hdhd5"         "Cyp4f13"      
[205] "Cyp2j9"        "Cideb"         "Gm56959"       "Acot11"       
[209] "Nat8f5"        "Fgd6"          "Arhgef19"      "Kcnj16"       
[213] "Nim1k"         "Glud1"         "Ldlr"          "Ptch1"        
[217] "Gas1"          "4930488L21Rik" "Nphp3"         "Gm43622"      
[221] "Tspan7"        "Cbr3"          "Ttpa"          "Slc1a4"       
[225] "Slc14a1"       "Nat8f3"        "Klf15"         "Elovl2"       
[22

#### 1 vs. mean tests

In [7]:
mean_res_list <- readRDS("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_1_vs_meanOthers_DE_genes_dream.RDS")
mean_ctype_genes <- prep_DE_genes(mean_res_list, lfc_threshold, pairwise=FALSE, unique=unique)

In [8]:
mean_ctype_genes

$Astro
  [1] "Gpr37l1"       "Slco1c1"       "Ntsr2"         "Slc7a10"      
  [5] "Gja1"          "Slc1a3"        "Cldn10"        "S1pr1"        
  [9] "Gjb6"          "Clu"           "Slc1a2"        "Aqp4"         
 [13] "Gm6145"        "Bcan"          "Lcat"          "Atp1a2"       
 [17] "Slc39a12"      "Plpp3"         "Apoe"          "F3"           
 [21] "Acsbg1"        "Fgfr3"         "Mfge8"         "Mt1"          
 [25] "Sncb"          "Slc38a3"       "Ptprz1"        "Cyp4f15"      
 [29] "Timp4"         "Aldoc"         "Gm3764"        "Anks1b"       
 [33] "Mlc1"          "Htra1"         "Glul"          "Scg3"         
 [37] "Btbd17"        "Pla2g7"        "Cspg5"         "Dio2"         
 [41] "Slc6a1"        "Sox9"          "Slc25a18"      "Phkg1"        
 [45] "Mt2"           "Tceal6"        "Fam107a"       "Kcnk1"        
 [49] "Myt1l"         "Ttyh1"         "Nwd1"          "Cmtm5"        
 [53] "Prdx6"         "Snap25"        "App"           "Cbs"          
 [57] "Caly"          "Sult4a1"       "Acsl3"         "Ly6h"         
 [61] "B230209E15Rik" "Tmem59l"       "Syt1"          "Ckmt1"        
 [65] "Prex2"         "S100a16"       "Msmo1"         "Ndrg2"        
 [69] "Hepacam"       "Prodh"         "Atp13a4"       "Mal2"         
 [73] "Syngr3"        "Nap1l2"        "Dbx2"          "Itih3"        
 [77] "Oaf"           "Grin2c"        "Uchl1"         "Eva1a"        
 [81] "Gstm1"         "Basp1"         "Gabrg2"        "Igfbp2"       
 [85] "Id4"           "Sv2a"          "Nsg2"          "Ptprn"        
 [89] "Ppp1r3g"       "Bex1"          "Fxyd1"         "Fabp3"        
 [93] "Etnppl"        "Tril"          "Mertk"         "Dtna"         
 [97] "Nherf1"        "Ntrk2"         "Slc4a10"       "Vegfa"        
[101] "Gm45774"       "Tmsb10"        "Eef1a2"        "Tceal5"       
[105] "Sox2"          "Tsc22d4"       "Chgb"          "Ina"          
[109] "Cd200"         "Dusp26"        "Ube2ql1"       "Chga"         
[113] "Acsl6"         "Fam20a"        "Fabp7"         "Opcml"        
[117] "Map1b"         "Slc4a4"        "Chrdl1"        "Plcd4"        
[121] "Dync1i1"       "Daam2"         "Myorg"         "Arhgdig"      
[125] "Vcam1"         "Mfsd2a"        "Kcnn2"         "Selenop"      
[129] "Nkain4"        "AW047730"      "Elmod1"        "Tmem130"      
[133] "Atcay"         "Ank3"          "Gabrb1"        "Dnm1"         
[137] "Rtn1"          "Rorb"          "Rmst"          "Plppr4"       
[141] "Cyp2j9"        "B3galnt1"      "Gldc"          "Dnajc6"       
[145] "Entpd2"        "Stmn3"         "Smim18"        "Rian"         
[149] "Slc27a1"       "Ica1"          "Gng2"          "Snhg11"       
[153] "Serp2"         "Coro1a"        "Sh3gl2"        "Carmil2"      
[157] "Syt4"          "Rit2"          "Mmd2"          "Meg3"         
[161] "Gng3"          "Kcnj10"        "Pak3"          "Stmn2"        
[165] "Scn2a"         "Nmnat2"        "Prkar1b"       "St8sia3"      
[169] "Syt13"         "Lfng"          "Disp2"         "Cdh19"        
[173] "Slc15a2"       "Tubb3"         "Gria4"         "Ednrb"        
[177] "Ly6e"          "Rfx4"          "Sez6l2"        "Clstn3"       
[181] "Vsnl1"         "Mir124a-1hg"   "Hprt1"         "Adhfe1"       
[185] "Cyp2d22"       "Thrsp"         "Sdc4"          "C1qtnf4"      
[189] "Smpdl3a"       "Scn2b"         "Syn1"          "Slc6a11"      
[193] "Slc12a5"       "Clmn"          "Lrp11"         "Cacng2"       
[197] "2900052N01Rik" "S100a1"        "Appl2"         "Luzp2"        
[201] "Scrg1"         "Bsn"           "Srcin1"        "Chd5"         
[205] "Nat8f1"        "Bend6"         "Csrp1"         "Pianp"        
[209] "Csrnp3"        "Matk"          "Bex4"          "Kcnq2"        
[213] "Snap91"        "Phka1"         "Atp1b2"        "Qki"          
[217] "Gpc5"          "Hacd2"         "Atp9a"         "Gramd2b"      
[221] "Clec2l"        "Stmn4"         "Ttc9b"         "Gli3"         
[225] "Atp1a3"        "Fermt2"        "Hes5"          "Rph3a"        
[22

### Enrichment results using 1 vs. pooled DE genes

In [9]:
pooled_enrichments_df <- get_module_enrichments(network_dir, pooled_ctype_genes, mod_def)

In [10]:
# pooled_enrichments_df %>%
#     group_by(Cell_type) %>%
#     slice_min(Qval) %>%
#     group_by(Cell_type) %>%
#     arrange(Network, Module) %>%
#     arrange(Qval)

In [11]:
# Get most enriched cell type for each module
# If cell type is most enriched in multiple modules, choose module with smallest p-value

pooled_top_mods_df <- pooled_enrichments_df %>%
    group_by(Network, Module) %>%
    slice_min(Qval) %>%
    group_by(Cell_type) %>%
    arrange(Network) %>%
    slice_min(Qval, with_ties=FALSE) %>%
    arrange(Qval)

In [12]:
pooled_top_mods_df[,c("Cell_type", "Pval", "Qval", "Module", "Network")]

Cell_type,Pval,Qval,Module,Network
<chr>,<dbl>,<dbl>,<chr>,<chr>
Endo,3.466417e-199,4.497433e-194,lightyellow,Bicor-None_signum0.803_minSize4_merge_ME_0.9_20151
Astro,6.587643e-185,1.709401e-180,blue,Bicor-None_signum0.24_minSize8_merge_ME_0.9_20151
Macrophage,3.011969e-168,2.442387e-164,pink,Bicor-None_signum0.803_minSize6_merge_ME_0.9_20151
Oligo,1.179014e-131,3.642113e-128,red,Bicor-None_signum0.89_minSize3_merge_ME_0.9_20151
VLMC,2.207749e-100,5.404527e-97,lightgreen,Bicor-None_signum0.361_minSize8_merge_ME_0.9_20151
SMC,4.518645e-90,9.610862e-87,blue,Bicor-None_signum0.89_minSize6_merge_ME_0.9_20151
Peri,1.400985e-64,2.042337e-61,brown,Bicor-None_signum0.803_minSize10_merge_ME_0.9_20151
Vip,6.127253e-58,7.643925e-55,brown,Bicor-None_signum0.214_minSize5_merge_ME_0.9_20151
L5_IT,1.191631e-50,1.198494e-47,turquoise,Bicor-None_signum0.282_minSize5_merge_ME_0.9_20151


In [13]:
if (unique) {
    write.csv(pooled_top_mods_df, file=paste0("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_1_vs_pooled_DE_genes_dream_unique_", network_dataset, "_", mod_def, "_top_Qval_modules.csv"), row.names=FALSE, quote=FALSE)
} else {
    write.csv(pooled_top_mods_df, file=paste0("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_1_vs_pooled_DE_genes_dream_", network_dataset, "_", mod_def, "_top_Qval_modules.csv"), row.names=FALSE, quote=FALSE)
}

### Enrichment results using pairwise DE genes

In [14]:
pairwise_enrichments_df <- get_module_enrichments(network_dir, pairwise_ctype_genes, mod_def)

In [15]:
# Get most enriched cell type for each module
# If cell type is most enriched in multiple modules, choose module with smallest p-value

pairwise_top_mods_df <- pairwise_enrichments_df %>%
    group_by(Network, Module) %>%
    slice_min(Qval) %>%
    group_by(Cell_type) %>%
    arrange(Network) %>%
    slice_min(Qval, with_ties=FALSE) %>%
    arrange(Qval)

In [16]:
pairwise_top_mods_df[,c("Cell_type", "Pval", "Qval", "Module", "Network")]

Cell_type,Pval,Qval,Module,Network
<chr>,<dbl>,<dbl>,<chr>,<chr>
Astro,0.000000e+00,0.000000e+00,blue,Bicor-None_signum0.214_minSize10_merge_ME_0.9_20151
Endo,0.000000e+00,0.000000e+00,black,Bicor-None_signum0.627_minSize10_merge_ME_0.9_20151
Macrophage,0.000000e+00,0.000000e+00,pink,Bicor-None_signum0.803_minSize5_merge_ME_0.9_20151
Oligo,6.755222e-236,1.991915e-232,red,Bicor-None_signum0.89_minSize3_merge_ME_0.9_20151
VLMC,2.818676e-186,6.772286e-183,lightgreen,Bicor-None_signum0.361_minSize8_merge_ME_0.9_20151
Pvalb,1.762940e-125,2.486186e-122,brown,Bicor-None_signum0.627_minSize10_merge_ME_0.9_20151
SMC,2.959107e-110,3.839234e-107,lightcyan,Bicor-None_signum0.361_minSize10_merge_ME_0.9_20151
Sst,2.200926e-74,1.720210e-71,tan,Bicor-None_signum0.89_minSize3_merge_ME_0.9_20151
Lamp5,1.607808e-69,1.115518e-66,magenta,Bicor-None_signum0.803_minSize5_merge_ME_0.9_20151


In [17]:
if (unique) {
    write.csv(pairwise_top_mods_df, file=paste0("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_pairwise_DE_genes_dream_unique_", network_dataset, "_", mod_def, "_top_Qval_modules.csv"), row.names=FALSE, quote=FALSE)
} else {
    write.csv(pairwise_top_mods_df, file=paste0("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_pairwise_DE_genes_dream_", network_dataset, "_", mod_def, "_top_Qval_modules.csv"), row.names=FALSE, quote=FALSE)
}

### Enrichment results using 1 vs. mean DE genes

In [18]:
mean_enrichments_df <- get_module_enrichments(network_dir, mean_ctype_genes, mod_def)

In [19]:
# Get most enriched cell type for each module
# If cell type is most enriched in multiple modules, choose module with smallest p-value

mean_top_mods_df <- mean_enrichments_df %>%
    group_by(Network, Module) %>%
    slice_min(Qval) %>%
    group_by(Cell_type) %>%
    arrange(Network) %>%
    slice_min(Qval, with_ties=FALSE) %>%
    arrange(Qval)

In [20]:
mean_top_mods_df[,c("Cell_type", "Pval", "Qval", "Module", "Network")]

Cell_type,Pval,Qval,Module,Network
<chr>,<dbl>,<dbl>,<chr>,<chr>
Astro,2.591508e-246,3.362301e-241,blue,Bicor-None_signum0.24_minSize10_merge_ME_0.9_20151
Macrophage,1.824211e-240,5.916966e-236,pink,Bicor-None_signum0.803_minSize6_merge_ME_0.9_20151
Endo,4.878059e-231,5.753581e-227,lightyellow,Bicor-None_signum0.803_minSize4_merge_ME_0.9_20151
Oligo,2.919049e-185,1.023584e-181,red,Bicor-None_signum0.89_minSize3_merge_ME_0.9_20151
VLMC,5.018046e-132,1.302113e-128,lightgreen,Bicor-None_signum0.361_minSize8_merge_ME_0.9_20151
L5_IT,1.500448e-124,3.539503e-121,turquoise,Bicor-None_signum0.282_minSize5_merge_ME_0.9_20151
SMC,1.208521e-119,2.570445e-116,blue,Bicor-None_signum0.89_minSize6_merge_ME_0.9_20151
Vip,5.854681e-76,5.585322e-73,red,Bicor-None_signum0.803_minSize6_merge_ME_0.9_20151
Lamp5,8.858910e-70,7.818922e-67,black,Bicor-None_signum0.89_minSize3_merge_ME_0.9_20151


In [21]:
if (unique) {
    write.csv(mean_top_mods_df, file=paste0("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_1_vs_meanOthers_DE_genes_dream_unique_", network_dataset, "_", mod_def, "_top_Qval_modules.csv"), row.names=FALSE, quote=FALSE)
} else {
    write.csv(mean_top_mods_df, file=paste0("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_1_vs_meanOthers_DE_genes_dream_", network_dataset, "_", mod_def, "_top_Qval_modules.csv"), row.names=FALSE, quote=FALSE)
}

### Compare enrichment results

In [22]:
pooled_top_mods_df$Network_short <- gsub("Bicor-None_", "", gsub("_merge_ME_0.9_20151", "", pooled_top_mods_df$Network))
pairwise_top_mods_df$Network_short <- gsub("Bicor-None_", "", gsub("_merge_ME_0.9_20151", "", pairwise_top_mods_df$Network))
mean_top_mods_df$Network_short <- gsub("Bicor-None_", "", gsub("_merge_ME_0.9_20151", "", mean_top_mods_df$Network))

cols <- c("Cell_type", "Qval", "Module", "Network_short")

all_top_mods_df <- merge(
    merge(
        pooled_top_mods_df[,cols], 
        pairwise_top_mods_df[,cols], 
        by="Cell_type", all=TRUE
    ), 
    mean_top_mods_df[,cols], by="Cell_type", all=TRUE
)

In [23]:
all_top_mods_df <- all_top_mods_df[,order(colnames(all_top_mods_df))]
all_top_mods_df

Cell_type,Module,Module.x,Module.y,Network_short,Network_short.x,Network_short.y,Qval,Qval.x,Qval.y
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
Astro,blue,blue,blue,signum0.24_minSize10,signum0.24_minSize8,signum0.214_minSize10,3.362301e-241,1.709401e-180,0.000000e+00
CR,bisque,paleturquoise,mediumvioletred,signum0.214_minSize5,signum0.627_minSize5,signum0.214_minSize5,4.202336e-03,5.373391e-11,5.817233e-06
Endo,lightyellow,lightyellow,black,signum0.803_minSize4,signum0.803_minSize4,signum0.627_minSize10,5.753581e-227,4.497433e-194,0.000000e+00
L2_3_IT,darkgrey,darkgrey,darkgrey,signum0.282_minSize10,signum0.282_minSize10,signum0.282_minSize10,6.058885e-34,4.218200e-11,8.862235e-23
L4,firebrick4,bisque4,skyblue,signum0.282_minSize10,signum0.361_minSize8,signum0.627_minSize6,2.927902e-02,3.403738e-03,2.269755e-01
L5_IT,turquoise,turquoise,turquoise,signum0.282_minSize5,signum0.282_minSize5,signum0.24_minSize10,3.539503e-121,1.198494e-47,3.004219e-60
L5_PT,sienna3,skyblue4,skyblue4,signum0.282_minSize10,signum0.627_minSize4,signum0.627_minSize4,2.660116e-22,1.605205e-16,1.634177e-16
L6_CT,floralwhite,magenta4,lightpink2,signum0.803_minSize3,signum0.361_minSize10,signum0.282_minSize6,7.551418e-36,7.434814e-41,1.245609e-27
L6_IT,blue2,blue2,blue2,signum0.627_minSize4,signum0.627_minSize4,signum0.627_minSize4,2.976385e-45,3.397525e-27,1.029830e-23


### Select the module from the lowest Qval per cell type

In [24]:
cols <- c("Cell_type", "Qval", "Module", "Network_short")

In [25]:
# Select the module from the lowest Qval per cell type

pooled_top_mods_df$DE_Test <- "Pooled"
pairwise_top_mods_df$DE_Test <- "Pairwise"
mean_top_mods_df$DE_Test <- "Mean"

all_top_mods_df <- rbind(pooled_top_mods_df, pairwise_top_mods_df, mean_top_mods_df)

In [26]:
top_qval_mods_df <- all_top_mods_df %>%
    group_by(Cell_type) %>%
    slice_min(Qval, with_ties=FALSE)

top_qval_mods_df[, c(cols, "DE_Test")] %>% arrange(Qval)

Cell_type,Qval,Module,Network_short,DE_Test
<chr>,<dbl>,<chr>,<chr>,<chr>
Astro,0.000000e+00,blue,signum0.214_minSize10,Pairwise
Endo,0.000000e+00,black,signum0.627_minSize10,Pairwise
Macrophage,0.000000e+00,pink,signum0.803_minSize5,Pairwise
Oligo,1.991915e-232,red,signum0.89_minSize3,Pairwise
VLMC,6.772286e-183,lightgreen,signum0.361_minSize8,Pairwise
Pvalb,2.486186e-122,brown,signum0.627_minSize10,Pairwise
L5_IT,3.539503e-121,turquoise,signum0.282_minSize5,Mean
SMC,2.570445e-116,blue,signum0.89_minSize6,Mean
Vip,5.585322e-73,red,signum0.803_minSize6,Mean


In [27]:
top_qval_mods_df

Network,kME_path,ME_path,Pval,Cell_type,Module,Qval,Network_short,DE_Test
<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>
Bicor-None_signum0.214_minSize10_merge_ME_0.9_20151,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_30pcntVar_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.214_minSize10_merge_ME_0.9_20151/kME_table_03-31-25.csv,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_30pcntVar_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.214_minSize10_merge_ME_0.9_20151/Module_eigengenes_03-31-25.csv,0.000000e+00,Astro,blue,0.000000e+00,signum0.214_minSize10,Pairwise
Bicor-None_signum0.627_minSize5_merge_ME_0.9_20151,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_30pcntVar_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.627_minSize5_merge_ME_0.9_20151/kME_table_10-54-11.csv,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_30pcntVar_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.627_minSize5_merge_ME_0.9_20151/Module_eigengenes_10-54-11.csv,3.168298e-13,CR,paleturquoise,5.373391e-11,signum0.627_minSize5,Pooled
Bicor-None_signum0.627_minSize10_merge_ME_0.9_20151,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_30pcntVar_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.627_minSize10_merge_ME_0.9_20151/kME_table_11-00-09.csv,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_30pcntVar_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.627_minSize10_merge_ME_0.9_20151/Module_eigengenes_11-00-09.csv,0.000000e+00,Endo,black,0.000000e+00,signum0.627_minSize10,Pairwise
Bicor-None_signum0.282_minSize10_merge_ME_0.9_20151,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_30pcntVar_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.282_minSize10_merge_ME_0.9_20151/kME_table_12-53-35.csv,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_30pcntVar_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.282_minSize10_merge_ME_0.9_20151/Module_eigengenes_12-53-35.csv,1.858625e-36,L2_3_IT,darkgrey,6.058885e-34,signum0.282_minSize10,Mean
Bicor-None_signum0.361_minSize8_merge_ME_0.9_20151,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_30pcntVar_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.361_minSize8_merge_ME_0.9_20151/kME_table_11-50-55.csv,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_30pcntVar_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.361_minSize8_merge_ME_0.9_20151/Module_eigengenes_11-50-55.csv,8.025122e-05,L4,bisque4,3.403738e-03,signum0.361_minSize8,Pooled
Bicor-None_signum0.282_minSize5_merge_ME_0.9_20151,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_30pcntVar_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.282_minSize5_merge_ME_0.9_20151/kME_table_12-00-42.csv,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_30pcntVar_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.282_minSize5_merge_ME_0.9_20151/Module_eigengenes_12-00-42.csv,1.500448e-124,L5_IT,turquoise,3.539503e-121,signum0.282_minSize5,Mean
Bicor-None_signum0.282_minSize10_merge_ME_0.9_20151,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_30pcntVar_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.282_mi

In [28]:
if (unique) {
    write.csv(top_qval_mods_df, file=paste0("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_DE_genes_dream_unique_", network_dataset, "_", mod_def, "_top_Qval_modules.csv"), row.names=FALSE, quote=FALSE)
} else {
    write.csv(top_qval_mods_df, file=paste0("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_DE_genes_dream_", network_dataset, "_", mod_def, "_top_Qval_modules.csv"), row.names=FALSE, quote=FALSE)
}
